# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Business Problem section 

#### Background

According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales. 

#### Business Problem

In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario? 

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.




##  Data section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.


## Methodology section

The Methodology section will describe the main components of our analysis and predication system.
The Methodology section comprises four stages: 
    1. Collect Inspection Data
    2. Explore and Understand Data
    3. Data preparation and preprocessing 
    4. Modeling

#### 1. Collect Inspection Data

After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

#### 2. Explore and Understand Data

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows: 

In [3]:
df_ppd.head(5) 

,{6DA0844A-2DB9-30F2-E053-6B04A8C05F3B},597000,2018-05-04 00:00,W2 6BN,F,N,L,58B,Unnamed: 8,GLOUCESTER GARDENS,Unnamed: 10,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A.1
0,{6DA0844A-2DBA-30F2-E053-6B04A8C05F3B},3400000,2018-05-23 00:00,NW6 1HS,D,N,F,37,NaN,CREDITON HILL,NaN,LONDON,CAMDEN,GREATER LONDON,A,A
1,{6DA0844A-2DBB-30F2-E053-6B04A8C05F3B},431000,2018-04-17 00:00,SW1P 4HN,F,N,L,DUKES HOUSE,FLAT 17,VINCENT STREET,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A
2,{6DA0844A-2DBC-30F2-E053-6B04A8C05F3B},430000,2018-05-11 00:00,N5 2UA,F,N,L,50,NaN,HIGHBURY QUADRANT,NaN,LONDON,ISLINGTON,GREATER LONDON,A,A
3,{6DA0844A-2DBD-30F2-E053-6B04A8C05F3B},462000,2018-05-09 00:00,N19 4JR,F,N,L,73C,NaN,LANDSEER ROAD,NaN,LONDON,ISLINGTON,GREATER LONDON,A,A
4,{6DA0844A-2DBE-30F2-E053-6B04A8C05F3B},585000,2018-05-02 00:00,W12 0AD,T,N,F,204,NaN,WULFSTAN STREET,NaN,LONDON,HAMMERSMITH AND FULHAM,GREATER LONDON,A,A


In [4]:
df_ppd.shape

(726020, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly. 

#### 3. Data preparation and preprocessing

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps: 
- Rename the column names 
- Format the date column 
- Sort data by date of sale 
- Select data only for the city of London 
- Make a list of street names in London 
- Calculate the street-wise average price of the property
- Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names 
- Join the data to find the coordinates of locations which fit into client's budget
- Plot recommended locations on London map along with current market prices






In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [6]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [7]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [8]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [9]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [10]:
# Display the dataframe
df_affordable

,Street,Avg_Price
20,ABBOTSBURY CLOSE,2.367093e+06
178,ALBION SQUARE,2.450000e+06
355,ANHALT ROAD,2.435000e+06
368,ANSDELL TERRACE,2.250000e+06
381,APPLEGARTH ROAD,2.400000e+06
617,AYLESTONE AVENUE,2.286667e+06
753,BARONSMEAD ROAD,2.375000e+06
867,BEAUCLERC ROAD,2.480000e+06
1079,BICKENHALL STREET,2.351667e+06
1094,BILLING ROAD,2.200000e+06


In [11]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [12]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 20
item: Street       ABBOTSBURY CLOSE
Avg_Price         2.36709e+06
Name: 20, dtype: object
item.Street only: ABBOTSBURY CLOSE
index: 178
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 178, dtype: object
item.Street only: ALBION SQUARE
index: 355
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 355, dtype: object
item.Street only: ANHALT ROAD
index: 368
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 368, dtype: object
item.Street only: ANSDELL TERRACE
index: 381
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 381, dtype: object
item.Street only: APPLEGARTH ROAD
index: 617
item: Street       AYLESTONE AVENUE
Avg_Price         2.28667e+06
Name: 617, dtype: object
item.Street only: AYLESTONE AVENUE
index: 753
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 753, dtype: object
item.Street only: BARONSMEAD ROAD
index: 867
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Nam

In [13]:
geolocator = Nominatim()


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [14]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_affordable

,Street,Avg_Price,city_coord
20,ABBOTSBURY CLOSE,2.367093e+06,"(51.5322588, -0.0061531)"
178,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
355,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
368,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
381,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
617,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)"
753,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
867,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1079,BICKENHALL STREET,2.351667e+06,"(51.5211969, -0.1589341)"
1094,BILLING ROAD,2.200000e+06,"(51.4818833, -0.1878624)"


In [16]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [17]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
20,ABBOTSBURY CLOSE,2.367093e+06,"(51.5322588, -0.0061531)",51.532259,-0.006153
178,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
355,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
368,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
381,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
617,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)",51.540916,-0.217874
753,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
867,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1079,BICKENHALL STREET,2.351667e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1094,BILLING ROAD,2.200000e+06,"(51.4818833, -0.1878624)",51.481883,-0.187862


In [18]:
df = df_affordable.drop(columns=['city_coord'])


In [19]:
df

,Street,Avg_Price,Latitude,Longitude
20,ABBOTSBURY CLOSE,2.367093e+06,51.532259,-0.006153
178,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
355,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
368,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
381,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
617,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874
753,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
867,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1079,BICKENHALL STREET,2.351667e+06,51.521197,-0.158934
1094,BILLING ROAD,2.200000e+06,51.481883,-0.187862


In [20]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [21]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [22]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3
CLIENT_SECRET:QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

#### 4. Modeling

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ABBOTSBURY CLOSE
ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
AYLESTONE AVENUE
BARONSMEAD ROAD
BEAUCLERC ROAD
BICKENHALL STREET
BILLING ROAD
BIRCHLANDS AVENUE
BOWERDEAN STREET
BRAMPTON GROVE
BRIARDALE GARDENS
BROWNING CLOSE
CALLCOTT STREET
CAMPDEN HILL ROAD
CANFIELD GARDENS
CANNING PLACE
CARLISLE ROAD
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHISWICK MALL
CLARENDON STREET
CLEVELAND SQUARE
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRAVEN STREET
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DRAX AVENUE
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
FIRECREST DRIVE
FLASK WALK
FRANK DIXON WAY
GERARD ROAD
GIBSON SQUARE
GLOUCESTER AVENUE
GORDON PLACE
HAMPSTEAD WAY
HARMAN DRIVE
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HILLGATE PLACE
HOLDEN AVENUE
HOLLYWOOD MEWS
HORNTON STREET
HUNTER ROAD
JACKSONS LANE
JOHN ISLIP STREET
KNOX STREET
LANCASTER MEWS
LAXTON PLA

In [25]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ABBOTSBURY CLOSE,51.532259,-0.006153,Pie Crust Cafe,51.536489,-0.004051,Thai Restaurant
1,ABBOTSBURY CLOSE,51.532259,-0.006153,Three Mills Green,51.528840,-0.006834,Park
2,ABBOTSBURY CLOSE,51.532259,-0.006153,Tesco Express,51.535118,-0.005973,Grocery Store
3,ABBOTSBURY CLOSE,51.532259,-0.006153,Holiday Inn Express,51.536332,-0.004623,Hotel
4,ABBOTSBURY CLOSE,51.532259,-0.006153,Bow Riviera,51.528684,-0.010352,Waterfront
5,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
6,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
7,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
8,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
9,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café


In [26]:
location_venues.groupby('Street').count()


,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ABBOTSBURY CLOSE,5,5,5,5,5,5
ALBION SQUARE,27,27,27,27,27,27
ANHALT ROAD,14,14,14,14,14,14
ANSDELL TERRACE,56,56,56,56,56,56
APPLEGARTH ROAD,5,5,5,5,5,5
AYLESTONE AVENUE,5,5,5,5,5,5
BARONSMEAD ROAD,13,13,13,13,13,13
BEAUCLERC ROAD,30,30,30,30,30,30
BICKENHALL STREET,95,95,95,95,95,95


In [27]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))


There are 313 uniques categories.


In [28]:
location_venues.shape

(4869, 7)

In [29]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ABBOTSBURY CLOSE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABBOTSBURY CLOSE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ABBOTSBURY CLOSE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ABBOTSBURY CLOSE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ABBOTSBURY CLOSE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [30]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ABBOTSBURY CLOSE,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0
1,ALBION SQUARE,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.037037,0.000000,...,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,ANHALT ROAD,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,ANSDELL TERRACE,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.017857,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.0
4,APPLEGARTH ROAD,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,AYLESTONE AVENUE,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,BARONSMEAD ROAD,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,BEAUCLERC ROAD,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,BICKENHALL STREET,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010526,0.000000,0.000000,0.010526,0.0
9,BILLING ROAD,0.00,0.000000,0.000000,0.0,0.000000,0.015873,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [31]:
london_grouped.shape


(125, 314)

In [32]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABBOTSBURY CLOSE----
             venue  freq
0  Thai Restaurant   0.2
1             Park   0.2
2       Waterfront   0.2
3    Grocery Store   0.2
4            Hotel   0.2


----ALBION SQUARE----
               venue  freq
0               Café  0.22
1                Bar  0.07
2                Pub  0.07
3  Indian Restaurant  0.07
4        Coffee Shop  0.07


----ANHALT ROAD----
                  venue  freq
0                   Pub  0.29
1   Japanese Restaurant  0.07
2  Gym / Fitness Center  0.07
3           Pizza Place  0.07
4                 Diner  0.07


----ANSDELL TERRACE----
                venue  freq
0      Clothing Store  0.07
1  Italian Restaurant  0.07
2           Juice Bar  0.05
3               Hotel  0.05
4   Indian Restaurant  0.05


----APPLEGARTH ROAD----
                     venue  freq
0                      Bar   0.4
1                Nightclub   0.2
2                   Casino   0.2
3                      Pub   0.2
4  New American Restaurant   0.0


----AYLESTONE AVE

In [33]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))



In [35]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)



In [36]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABBOTSBURY CLOSE,Grocery Store,Park,Waterfront,Hotel,Thai Restaurant,Farm,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space
1,ALBION SQUARE,Café,Restaurant,Indian Restaurant,Bar,Coffee Shop,Pub,New American Restaurant,Seafood Restaurant,Fish & Chips Shop,Brewery
2,ANHALT ROAD,Pub,Plaza,Pizza Place,Grocery Store,Japanese Restaurant,French Restaurant,English Restaurant,Gym / Fitness Center,Diner,Garden
3,ANSDELL TERRACE,Clothing Store,Italian Restaurant,Café,English Restaurant,Pub,Juice Bar,Hotel,Indian Restaurant,Bakery,Garden
4,APPLEGARTH ROAD,Bar,Pub,Casino,Nightclub,Fast Food Restaurant,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm


In [37]:
venues_sorted.shape

(125, 11)

In [38]:
london_grouped.shape

(125, 314)

In [39]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby. 

In [40]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([2, 0, 4, 1, 4, 3, 2, 0, 2, 1, 1, 3, 0, 4, 3, 2, 2, 3, 4, 1, 3, 3,
       4, 0, 0, 1, 3, 1, 4, 4, 2, 1, 2, 2, 4, 4, 4, 3, 1, 4, 3, 2, 3, 1,
       2, 4, 1, 1, 1, 1], dtype=int32)

In [41]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
20,ABBOTSBURY CLOSE,2.367093e+06,51.532259,-0.006153
178,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
355,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
368,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
381,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670


In [42]:
london_grouped_clustering.shape

(131, 4)

In [43]:
df.shape

(131, 4)

In [44]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [45]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [46]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,ABBOTSBURY CLOSE,2.367093e+06,51.532259,-0.006153,2,Grocery Store,Park,Waterfront,Hotel,Thai Restaurant,Farm,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space
178,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,0,Café,Restaurant,Indian Restaurant,Bar,Coffee Shop,Pub,New American Restaurant,Seafood Restaurant,Fish & Chips Shop,Brewery
355,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,4,Pub,Plaza,Pizza Place,Grocery Store,Japanese Restaurant,French Restaurant,English Restaurant,Gym / Fitness Center,Diner,Garden
368,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,1,Clothing Store,Italian Restaurant,Café,English Restaurant,Pub,Juice Bar,Hotel,Indian Restaurant,Bakery,Garden
381,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,4,Bar,Pub,Casino,Nightclub,Fast Food Restaurant,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
617,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874,3,Park,Movie Theater,Bus Stop,Café,Fish & Chips Shop,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
753,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Food & Drink Shop,Restaurant,Breakfast Spot,Nature Preserve,Bookstore,Farmers Market,Thai Restaurant,Park,Coffee Shop,Pizza Place
867,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,0,Coffee Shop,Pub,Hotel,Thai Restaurant,Grocery Store,Chinese Restaurant,Cocktail Bar,Gastropub,Ice Cream Shop,Fish & Chips Shop
1079,BICKENHALL STREET,2.351667e+06,51.521197,-0.158934,2,Hotel,Chinese Restaurant,Gastropub,Restaurant,Italian Restaurant,Pizza Place,Pub,Café,Thai Restaurant,Greek Restaurant
1094,BILLING ROAD,2.200000e+06,51.481883,-0.187862,1,Pub,Soccer Stadium,Italian Restaurant,Grocery Store,Park,Gym / Fitness Center,Lounge,Gastropub,Café,Sandwich Place


In [47]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [48]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
178,2.450000e+06,Café,Restaurant,Indian Restaurant,Bar,Coffee Shop,Pub,New American Restaurant,Seafood Restaurant,Fish & Chips Shop,Brewery
867,2.480000e+06,Coffee Shop,Pub,Hotel,Thai Restaurant,Grocery Store,Chinese Restaurant,Cocktail Bar,Gastropub,Ice Cream Shop,Fish & Chips Shop
1371,2.475833e+06,Bar,Lake,Middle Eastern Restaurant,Men's Store,Fish & Chips Shop,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
2237,2.495000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2278,2.450000e+06,Garden,Cocktail Bar,Gym / Fitness Center,Park,Beer Bar,Buffet,Bar,Coffee Shop,Mexican Restaurant,Performing Arts Venue


In [49]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
368,2250000.0,Clothing Store,Italian Restaurant,Café,English Restaurant,Pub,Juice Bar,Hotel,Indian Restaurant,Bakery,Garden
1094,2200000.0,Pub,Soccer Stadium,Italian Restaurant,Grocery Store,Park,Gym / Fitness Center,Lounge,Gastropub,Café,Sandwich Place
1108,2217000.0,Pub,French Restaurant,Train Station,Bakery,Lake,Coffee Shop,Chinese Restaurant,Flower Shop,Electronics Store,English Restaurant
1950,2200000.0,Nightlife Spot,Soccer Field,Fish & Chips Shop,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2306,2250000.0,Gastropub,Construction & Landscaping,Health & Beauty Service,Fish & Chips Shop,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market


In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,2.367093e+06,Grocery Store,Park,Waterfront,Hotel,Thai Restaurant,Farm,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space
753,2.375000e+06,Food & Drink Shop,Restaurant,Breakfast Spot,Nature Preserve,Bookstore,Farmers Market,Thai Restaurant,Park,Coffee Shop,Pizza Place
1079,2.351667e+06,Hotel,Chinese Restaurant,Gastropub,Restaurant,Italian Restaurant,Pizza Place,Pub,Café,Thai Restaurant,Greek Restaurant
1820,2.375000e+06,Pub,Bakery,Park,Hotel,Pizza Place,Indian Restaurant,Yoga Studio,Grocery Store,Hostel,Seafood Restaurant
1871,2.352889e+06,Pub,Bakery,Pizza Place,Hotel,Grocery Store,Indian Restaurant,Yoga Studio,Hostel,Farmers Market,Movie Theater


In [51]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
617,2.286667e+06,Park,Movie Theater,Bus Stop,Café,Fish & Chips Shop,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
1310,2.300000e+06,Coffee Shop,Pub,Italian Restaurant,Mediterranean Restaurant,Park,Thai Restaurant,Café,Furniture / Home Store,Gastropub,Chinese Restaurant
1605,2.320000e+06,Bakery,Gastropub,Fish & Chips Shop,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1889,2.278000e+06,Coffee Shop,Café,Italian Restaurant,Bookstore,Japanese Restaurant,Grocery Store,Pizza Place,Hotel,Supermarket,Sandwich Place
1963,2.300000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
355,2435000.0,Pub,Plaza,Pizza Place,Grocery Store,Japanese Restaurant,French Restaurant,English Restaurant,Gym / Fitness Center,Diner,Garden
381,2400000.0,Bar,Pub,Casino,Nightclub,Fast Food Restaurant,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
1439,2397132.0,Food Truck,Indian Restaurant,Grocery Store,Gym / Fitness Center,Coffee Shop,Farmers Market,Electronics Store,English Restaurant,Event Space,Exhibit
1894,2425000.0,Hotel,Indian Restaurant,French Restaurant,Chinese Restaurant,Italian Restaurant,Pub,Clothing Store,Bar,Park,Garden
2171,2414000.0,Coffee Shop,Bus Station,Zoo,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


## Results and Discussion section

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives. 

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer. 


## Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario? 

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments. 

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer. 

